In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from.env file
load_dotenv(".env")

# You can now access the API key using os.getenv
# openai_api_key = os.getenv("OPENAI_API_KEY")


True

In [5]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack.dataclasses import Document
from dotenv import load_dotenv
import os

load_dotenv()

# --- Prepare Data and Components ---

# We'll use some simple documents for this example
documents_for_pipeline = [
    Document(
        content="Rose Island was a micronation located in the Adriatic Sea. It declared independence in 1968 and had Italian as its official language.",
        meta={"source": "history_docs", "author": "historian1"},
    ),
    Document(
        content="The capital of Rose Island was called 'Isola delle Rose'. It was known for its unique architecture and vibrant culture.",
        meta={"source": "geography_docs", "author": "geographer1"},
    ),
    Document(
        content="Rose Island's economy was primarily based on tourism and the sale of souvenirs to visitors.",
        meta={"source": "economy_docs", "author": "economist1"},
    ),
]

# Define the prompt template again
prompt_template_for_pipeline = """
Answer the question based on this context.

Context:
{% for doc in documents %}
- {{ doc.content }}
{% endfor %}

Question: {{ query }}
Answer:
"""

# Instantiate the components we will use
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")


# --- Build the Pipeline ---

# 1. Instantiate the Pipeline
basic_pipeline = Pipeline()

# 2. Add Component Instances
# We give each component a unique name: "prompter" and "llm".
basic_pipeline.add_component(name="prompter", instance=prompt_builder_inst)
basic_pipeline.add_component(name="llm", instance=llm_generator_inst)

# 3. Connect the Components
# This is the crucial step that defines the data flow.
# We connect the 'prompt' output of the 'prompter' component
# to the 'prompt' input of the 'llm' component.
basic_pipeline.connect("prompter.prompt", "llm.prompt")


# --- Run the Pipeline ---

# The.run() method takes a dictionary as input.
# The keys of the dictionary correspond to the names of the components in the pipeline.
# The values are dictionaries of the inputs for that component.
query_text = "What was the official language of Rose Island?"

run_data = {
    "prompter": {
        "query": query_text,
        "documents": documents_for_pipeline
    }
}

# Execute the pipeline
pipeline_result = basic_pipeline.run(run_data)

# The output is a dictionary with the final results from the terminal component(s).
print("Pipeline Result:")
print(pipeline_result)




Pipeline Result:
{'llm': {'replies': ['The official language of Rose Island was Italian.'], 'meta': [{'model': 'gpt-4o-mini-2024-07-18', 'index': 0, 'finish_reason': 'stop', 'usage': {'completion_tokens': 9, 'prompt_tokens': 103, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}}]}}


In [7]:
basic_pipeline.draw(path="./images/prompt_pipeline.png")

print("\nPipeline visualization saved to './images/prompt_pipeline.png'")



Pipeline visualization saved to './images/prompt_pipeline.png'


![](./images/prompt_pipeline.png)